In [11]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 968.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 78.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.4/123.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 67.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [15]:
import boto3
import librosa
import math
import os
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

In [ ]:
#how to access files on s3

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
bucket = s3.Bucket("music-wav-charts")

i = 0
for obj in bucket.objects.all(): #resource
    if(obj.key[len(obj.key)-3:]) == "png":
        print(obj.key)
        i+=1
    #file = s3_client.download_file('music-wav-files', obj.key, 'audio.wav')
    #audio, sfreq = librosa.load('audio.wav')
    #print(audio)
print(i)

In [ ]:
#how to access files on s3

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
bucket = s3.Bucket("music-wav-files")

i = 0
for obj in bucket.objects.all(): #resource
    if(obj.key[len(obj.key)-3:]) == "wav":
        print(obj.key)
        i+=1
    #file = s3_client.download_file('music-wav-files', obj.key, 'audio.wav')
    #audio, sfreq = librosa.load('audio.wav')
    #print(audio)
print(i)

In [ ]:
#how to upload to json
my_dict = { 'Ali': 9, 'Sid': 1, 'Luna': 7, 'Sim': 12, 'Pooja': 4, 'Jen': 2}
with open('data.json', 'w') as fp:
    json.dump(my_dict, fp)

In [12]:
SAMPLE_RATE = 22050
DURATION = 30 #measured in seconds

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
bucket = s3.Bucket("music-wav-files")

SAMPLES_PER_TRACK = SAMPLE_RATE * DURATION
def save_mfcc(n_mfcc=13, n_fft=2048, 
              hop_length=512, num_segments = 5):
    
    with open('data.json', 'w') as fp:
        #dictionary to store data
        data = {
            "mapping": ["blues", "classical", "country", "disco",
                       "hiphop", "metal", "pop", "reggae", "rock"],
            "mfcc": [],
            "labels": []
        }

        num_samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
        expected_num_mfcc_vectors_per_seg = math.ceil(num_samples_per_segment / hop_length)
        j = 1
        k = 1
        for obj in bucket.objects.all(): #resource
            if j == 100:
                k+=1
                j = 1
                
            if(obj.key[len(obj.key)-3:]) == "wav":
                file = s3_client.download_file('music-wav-files', obj.key, 'audio.wav')
                #data["mapping"].append(obj.key)
                print("\nProcessing {}".format(obj.key))


                #load audio file
                signal, sr = librosa.load('audio.wav', sr=SAMPLE_RATE)

                #process segments extracting mfcc/storing data
                for s in range(num_segments):
                    start_sample = num_samples_per_segment * s
                    finish_sample = start_sample + num_samples_per_segment

                    mfcc = librosa.feature.mfcc(signal[start_sample:finish_sample],
                                               sr = sr,
                                               n_fft = n_fft,
                                               n_mfcc = n_mfcc,
                                               hop_length=hop_length
                                               )

                    mfcc = mfcc.T

                    #store mfcc for segment if it has the expected length

                    if len(mfcc) == expected_num_mfcc_vectors_per_seg:
                        data["mfcc"].append(mfcc.tolist())
                        data["labels"].append(k - 1)
                        print("{}, segment: {}".format(obj.key, s))

                j+=1


        json.dump(data, fp)

        return data


mfcc_dataset = save_mfcc(num_segments=10)
print("done processing")


Processing genres_original/blues/blues.00000.wav
genres_original/blues/blues.00000.wav, segment: 0
genres_original/blues/blues.00000.wav, segment: 1
genres_original/blues/blues.00000.wav, segment: 2
genres_original/blues/blues.00000.wav, segment: 3
genres_original/blues/blues.00000.wav, segment: 4


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[ 0.00732422  0.01660156  0.00762939 ... -0.04373169 -0.05706787
 -0.04086304] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[-0.02481079 -0.02005005 -0.0357666  ...  0.00918579  0.03964233
  0.09030151] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[ 0.06256104  0.02435303 -0.00405884 ...  0.2854004   0.29364014
  0.25228882] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: Fu

genres_original/blues/blues.00000.wav, segment: 5
genres_original/blues/blues.00000.wav, segment: 6
genres_original/blues/blues.00000.wav, segment: 7
genres_original/blues/blues.00000.wav, segment: 8
genres_original/blues/blues.00000.wav, segment: 9


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[-0.02093506 -0.05047607 -0.0357666  ...  0.24041748  0.2104187
  0.13391113] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[0.08325195 0.04660034 0.01455688 ... 0.05941772 0.11999512 0.14416504] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[ 0.1331482   0.08804321  0.02166748 ... -0.21609497 -0.23059082
 -0.20846558] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarn


Processing genres_original/blues/blues.00001.wav
genres_original/blues/blues.00001.wav, segment: 0
genres_original/blues/blues.00001.wav, segment: 1
genres_original/blues/blues.00001.wav, segment: 2
genres_original/blues/blues.00001.wav, segment: 3
genres_original/blues/blues.00001.wav, segment: 4
genres_original/blues/blues.00001.wav, segment: 5
genres_original/blues/blues.00001.wav, segment: 6
genres_original/blues/blues.00001.wav, segment: 7
genres_original/blues/blues.00001.wav, segment: 8


...


Processing genres_original/rock/rock.00098.wav
genres_original/rock/rock.00098.wav, segment: 0
genres_original/rock/rock.00098.wav, segment: 1
genres_original/rock/rock.00098.wav, segment: 2
genres_original/rock/rock.00098.wav, segment: 3
genres_original/rock/rock.00098.wav, segment: 4
genres_original/rock/rock.00098.wav, segment: 5
genres_original/rock/rock.00098.wav, segment: 6
genres_original/rock/rock.00098.wav, segment: 7
genres_original/rock/rock.00098.wav, segment: 8
genres_original/rock/rock.00098.wav, segment: 9


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[ 0.0859375   0.14483643  0.14294434 ... -0.14935303 -0.13977051
 -0.16055298] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[-0.18664551 -0.2218628  -0.25247192 ... -0.16693115 -0.15029907
 -0.14175415] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[-0.1418457  -0.13757324 -0.13110352 ...  0.06347656  0.06338501
  0.06719971] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: Fu


Processing genres_original/rock/rock.00099.wav
genres_original/rock/rock.00099.wav, segment: 0
genres_original/rock/rock.00099.wav, segment: 1
genres_original/rock/rock.00099.wav, segment: 2
genres_original/rock/rock.00099.wav, segment: 3
genres_original/rock/rock.00099.wav, segment: 4
genres_original/rock/rock.00099.wav, segment: 5
genres_original/rock/rock.00099.wav, segment: 6
genres_original/rock/rock.00099.wav, segment: 7
genres_original/rock/rock.00099.wav, segment: 8
genres_original/rock/rock.00099.wav, segment: 9


/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[-0.02111816 -0.03451538 -0.03536987 ...  0.01251221  0.01916504
  0.03027344] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[ 0.03543091  0.02877808  0.02859497 ... -0.0736084  -0.07369995
 -0.07391357] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: Pass y=[-0.0743103  -0.07473755 -0.07458496 ...  0.099823    0.11230469
  0.1378479 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.7/site-packages/ipykernel_launcher.py:48: Fu

done processing


In [ ]:
f = open('data.json')
data = json.load(f)

for a in data['mfcc']:
    print(a)

In [13]:
#load the dataset
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
    #convert lists into numpy arrays
    inputs = np.array(data["mfcc"])
    targets = np.array(data["labels"])
    
    return inputs, targets

if __name__ == "__main__":
    inputs, targets = load_data('data.json')

#split the data into training and test sets
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(
    inputs, targets, test_size = 0.3)

#build the network architecture
    model = keras.Sequential([
        #input layer
        keras.layers.Flatten(input_shape=(inputs.shape[1], inputs.shape[2])),
        
        #1st hidden layer
        keras.layers.Dense(512, activation="relu"),
        #keras.layers.Dropout(0.3),
        
        #2nd
        keras.layers.Dense(256, activation="relu"),
        #keras.layers.Dropout(0.3),
        #3rd
        keras.layers.Dense(64, activation="relu"),
        #keras.layers.Dropout(0.3),
        
        #output layer
        keras.layers.Dense(10, activation="softmax")
        
    ])

#compile the network
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])
    model.summary()

#train the network
    model.fit(inputs_train, targets_train, 
              validation_data=(inputs_test, targets_test),
              batch_size=32,
              epochs=50              
             )
    

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 1690)              0         
                                                                 
 dense_32 (Dense)            (None, 512)               865792    
                                                                 
 dense_33 (Dense)            (None, 256)               131328    
                                                                 
 dense_34 (Dense)            (None, 64)                16448     
                                                                 
 dense_35 (Dense)            (None, 900)               58500     
                                                                 
Total params: 1,072,068
Trainable params: 1,072,068
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
197/197 [========================

In [32]:
#load the dataset
def load_data(dataset_path):
    with open(dataset_path, "r") as fp:
        data = json.load(fp)
        
    X = np.array(data["mfcc"])
    Y = np.array(data["labels"])
    return X,Y #inputs, targets

def prepare_datasets(test_size, validation_size):
    #load data
    X, Y = load_data('data.json')
    
    #create the train/test split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)
    
    #create the train/validation split
    X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=validation_size)

    #3d array -> (130, 13, 1) see 12:00
    X_train = X_train[...,np.newaxis] #4d array -> (num_samples, 130, 13, 1)
    X_validation = X_validation[...,np.newaxis]
    X_test = X_test[...,np.newaxis]
    
    return X_train, X_validation, X_test, Y_train, Y_validation, Y_test
    
    
def build_model(input_shape):
    #create model
    model = keras.Sequential()
    
    #1st conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', 
                                 input_shape=input_shape))
    
    model.add(keras.layers.MaxPool2D( (3,3), strides=(2,2), padding='same' ))
    model.add(keras.layers.BatchNormalization())
    
    #2nd conv layer
    model.add(keras.layers.Conv2D(32, (3,3), activation='relu', 
                                 input_shape=input_shape))
    
    model.add(keras.layers.MaxPool2D( (3,3), strides=(2,2), padding='same' ))
    model.add(keras.layers.BatchNormalization())
    
    
    #3rd conv layer
    model.add(keras.layers.Conv2D(32, (2,2), activation='relu', 
                                 input_shape=input_shape))
    
    model.add(keras.layers.MaxPool2D( (2,2), strides=(2,2), padding='same' ))
    model.add(keras.layers.BatchNormalization())
    
    #flatten the output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    
    #output layer - could be 900 instead of 10
    model.add(keras.layers.Dense(10,activation='softmax'))
    
    return model

correct = 0

def predict(model, X, Y):
    global correct
    X = X[np.newaxis, ...]
    
    #prediction = [ [0.1, 0.2, ...] ]
    prediction = model.predict(X) #X -> (1, 130, 13, 1)
    
    #extract index with max value
    predicted_index = np.argmax(prediction, axis=1) # [3]
    print("Expected index: {}, Predicted index: {}".format(
    Y, predicted_index))
    
    if predicted_index == Y:
        correct+=1
    
    
if __name__ == "__main__":
    #create train, validation, and test sets
    X_train, X_validation, X_test, Y_train, Y_validation, Y_test = prepare_datasets(0.25, 0.2)
    #print(X_test.shape)
    #print(X_test[2248])
    #build the CNN network
    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    model = build_model(input_shape)
    
    #compile the network
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    #train the CNN
    model.fit(X_train, Y_train, validation_data=(X_validation, Y_validation),
              batch_size=32,
              epochs=30
             )
    
    #evaluate the CNN on the test set
    test_error, test_accuracy = model.evaluate(X_test, Y_test,
                                              verbose=1)
    print("Accuracy on test set is: {}".format(test_accuracy))
    
    
    #make predictions on a sample
    for i in range(100, 2248):
        X = X_test[i]
        Y = Y_test[i]
        predict(model, X,Y)
    print("Accuracy Rate: " + str(correct/2147) )



Epoch 1/30
169/169 [==============================] - 9s 52ms/step - loss: 2.4855 - accuracy: 0.2305 - val_loss: 1.9807 - val_accuracy: 0.3281
Epoch 2/30
169/169 [==============================] - 9s 51ms/step - loss: 1.9202 - accuracy: 0.3593 - val_loss: 1.6722 - val_accuracy: 0.4222
Epoch 3/30
169/169 [==============================] - 9s 52ms/step - loss: 1.7268 - accuracy: 0.4119 - val_loss: 1.5316 - val_accuracy: 0.4526
Epoch 4/30
169/169 [==============================] - 9s 53ms/step - loss: 1.5949 - accuracy: 0.4423 - val_loss: 1.4484 - val_accuracy: 0.4867
Epoch 5/30
169/169 [==============================] - 9s 51ms/step - loss: 1.5050 - accuracy: 0.4745 - val_loss: 1.3923 - val_accuracy: 0.5081
Epoch 6/30
169/169 [==============================] - 9s 51ms/step - loss: 1.4335 - accuracy: 0.4960 - val_loss: 1.3460 - val_accuracy: 0.5267
Epoch 7/30
169/169 [==============================] - 10s 61ms/step - loss: 1.3839 - accuracy: 0.5103 - val_loss: 1.3004 - val_accuracy: 0.543

In [23]:
x = 0
print("hello" + str(x/2))

hello0.0


In [30]:
correct = 0
def test():
    global correct
    correct+=1
    
test()
test()
print(correct)

2
